#Softmax Classification
- loss : Cross Entropy

** Binary Classification 일 경우, Binary Cross Entropy + Sigmoid 사용

** Multi Classification 일 경우, Cross Entropy + Softmax 사용


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

softmax를 통해 숫자를 확률로 변환한다.
$$ P(class=i) = \frac{e^i}{\sum e^i} $$

In [2]:
z = torch.FloatTensor([1,2,3])

In [3]:
hypothesis = F.softmax(z, dim = 0)
print(hypothesis)
print(hypothesis.sum())

tensor([0.0900, 0.2447, 0.6652])
tensor(1.)


## Cross Entropy
Multi-class 분류는 Cross Entropy를 Loss로 사용
$$ L = \frac{1}{N} \sum - y \log(\hat{y}) $$

In [4]:
z = torch.rand(3,5,requires_grad=True)  # classes = 5, samples = 3
hypothesis = F.softmax(z, dim = 1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [5]:
y = torch.randint(5,(3,)).long()
print(y)

tensor([0, 2, 1])


### Low-Level

In [6]:
y_one_hot = torch.zeros_like(hypothesis)
print(y.unsqueeze(1).shape)
y_one_hot.scatter_(1, y.unsqueeze(1),1)

torch.Size([3, 1])


tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [7]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


##Cross Entropy with torch.nn.functional

In [8]:
# Low level
torch.log(F.softmax(z, dim = 1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [9]:
# High level
F.log_softmax(z,dim = 1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

In [10]:
# Low level
(y_one_hot * -torch.log(F.softmax(z,dim = 1))).sum(dim = 1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

F.nll_loss() function that computes the negative loss likelihood

In [11]:
# High level
F.nll_loss(F.log_softmax(z,dim=1),y)

tensor(1.4689, grad_fn=<NllLossBackward>)

In [12]:
F.cross_entropy(z,y)

tensor(1.4689, grad_fn=<NllLossBackward>)

## Training with Low-level Cross Entropy Loss

In [13]:
# 데이터 셋
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2,2,2,1,1,1,0,0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [14]:
# 모델 초기화
W = torch.zeros((4,3),requires_grad=True)
b = torch.zeros(1,requires_grad=True)
# optimizer설정
optimizer = optim.SGD([W,b],lr= 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    #H(x) 계산
    hypo = F.softmax(x_train.matmul(W) + b, dim = 1)
    y_one_hot = torch.zeros_like(hypo)
    y_one_hot = y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
    # cost = (y_one_hot * -torch.log(F.softmax(hypo,dim=1))).sum(dim=1).mean()
    cost = (y_one_hot * -torch.log(hypo)).sum(dim=1).mean()
    # cost 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("Epoch: {:4d}/{} Cost: {:.6f}".format(epoch,nb_epochs,cost.item()))

Epoch:    0/1000 Cost: 1.098612
Epoch:  100/1000 Cost: 0.761050
Epoch:  200/1000 Cost: 0.689991
Epoch:  300/1000 Cost: 0.643229
Epoch:  400/1000 Cost: 0.604117
Epoch:  500/1000 Cost: 0.568255
Epoch:  600/1000 Cost: 0.533922
Epoch:  700/1000 Cost: 0.500291
Epoch:  800/1000 Cost: 0.466908
Epoch:  900/1000 Cost: 0.433507
Epoch: 1000/1000 Cost: 0.399962


## Training with High-level Cross Entropy

In [15]:
# 모델 초기화
W = torch.zeros((4,3), requires_grad= True)
b = torch.zeros(1,requires_grad= True)
# optimizer 설정
optimizer = optim.SGD([W,b],lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z,y_train)   # ont_hot vector 만드는 과정 생략

    # Cost 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("Epoch: {:4d}/{} Cost: {:.6f}".format(epoch,nb_epochs,cost.item()))

Epoch:    0/1000 Cost: 1.098612
Epoch:  100/1000 Cost: 0.761050
Epoch:  200/1000 Cost: 0.689991
Epoch:  300/1000 Cost: 0.643229
Epoch:  400/1000 Cost: 0.604117
Epoch:  500/1000 Cost: 0.568255
Epoch:  600/1000 Cost: 0.533922
Epoch:  700/1000 Cost: 0.500291
Epoch:  800/1000 Cost: 0.466908
Epoch:  900/1000 Cost: 0.433506
Epoch: 1000/1000 Cost: 0.399962


## High-level Implementation with nn.Module

In [16]:
class SoftmaxClassifiermodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3)
    
    def forward(self,x):
        return self.linear(x)       # x = (m,4) -> (m,3)

In [17]:
model = SoftmaxClassifiermodel()

In [18]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(),lr = 0.1)
nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    hypo = model(x_train)
    # cost 계산
    cost = F.cross_entropy(hypo,y_train)
    # cost로 h(x)개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
         print("Epoch: {:4d}/{} Cost: {:.6f}".format(epoch,nb_epochs,cost.item()))

Epoch:    0/1000 Cost: 1.849513
Epoch:  100/1000 Cost: 0.689894
Epoch:  200/1000 Cost: 0.609258
Epoch:  300/1000 Cost: 0.551218
Epoch:  400/1000 Cost: 0.500141
Epoch:  500/1000 Cost: 0.451947
Epoch:  600/1000 Cost: 0.405051
Epoch:  700/1000 Cost: 0.358733
Epoch:  800/1000 Cost: 0.312912
Epoch:  900/1000 Cost: 0.269522
Epoch: 1000/1000 Cost: 0.241922
